In [2]:
from sentence_transformers import SentenceTransformer
from tika import parser
from elasticsearch import Elasticsearch
import os

c:\Users\Lu Xuan Minh\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [7]:
# Choose SBERT model from https://sbert.net/docs/sentence_transformer/pretrained_models.html
model = SentenceTransformer('all-mpnet-base-v2')

c:\Users\Lu Xuan Minh\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
# Create connection to Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

In [15]:
directory = './data'

In [4]:
def index_files(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            # Extract raw text from file (maybe any file types)
            filepath = os.path.join(directory, filename)
            raw = parser.from_file(filepath)
            content = raw['content']
            
            if content:
                # Encode content into vector
                embedding = model.encode(content)
                
                # Indexing
                doc = {
                    'filename': filename,
                    'content': content,
                    'embedding': embedding.tolist()  # Convert numpy array to vector in order to store in Elasticsearch
                }
                es.index(index="files", body=doc)
                print(f"Indexed {filename}")

In [8]:
def search_query(query):
    # Encode query into vector
    query_vector = model.encode(query).tolist()
    
    query = {
        "field": "embedding",
        "query_vector": query_vector,
        "k": 3, # number of nearest neighbours
        "num_candidates": 10 # number of files must be traversed. This number maybe large in reality
    }
    
    res = es.knn_search(index="files", knn=query)
    return res['hits']['hits']

In [52]:
# Indexing files
index_files(directory)

Indexed 1.txt
Indexed 2.txt
Indexed 3.txt
Indexed 4.txt
Indexed 5.txt


In [9]:
query = "Artificial Intelligence (AI) has rapidly evolved over the past few decades"
results = search_query(query)

for result in results:
    print(f"Found file: {result['_source']['filename']} with score: {result['_score']}")

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [files]', files, index_or_alias)